# 自动站数据的类


In [4]:
%%time
import gevent
from math import isnan
import numpy as np
#import modin.pandas as pd 
import pandas as pd
import time
from func import *
import matplotlib
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import json
import math

import pymssql 

class zdz_data:
    def __init__(self, start,end):
        self.start = start
        self.end = end
        self.event_data(start,end)
        
        self.rain_data(start,end)
        
    def read_csv(self):
        '''
        1.数据读取---此方法为测试方法
        2.数据分段---每天
        3.每天统计
        '''
        station_Mws = pd.read_csv("Mws_15.csv")
        station_Aws = pd.read_csv("Aws_15.csv")
        station_all = pd.concat([station_Aws, station_Mws])
        return station_all
    def event_data(self,start,end):
        '''
        1.数据库查询
        2.数据分段---每天
        3.每天统计
        '''
        print('返回一个每天的数组和日历所需的数据')
    def return_daylist(self):
        '''
        返回每天的起始列表
        '''
        print('计算时间列表')
    def rain_data(self,start,end):
        '''
        1.根据起始时间计算面雨量的sql语句
        2.根据初始语句返回每站的总降水量
        '''
        print('降水页面的数据处理')
    def wind_data(self,start,end):
        '''
        1.根据sql语句计算8及以上大风的分布和排序
        '''
        print('风场的计算')
    def view_data(self,start,end):
        '''
        1.根据sql语句计算低能见度的分布和排序
        '''
        print('能见度的计算')
    def temp_data(,start,end):
        '''
        1.根据sql语句计算高低温的分布和排序
        '''
        print('温度的计算')
    def pre_day(self):
        '''
        日报的响应时间、触发后统计对应天的灾情
        '''
        print('计算指定日期的数据')


    

start =  'Wed Mar 01 2022 19:18:00 GMT+0800 (中国标准时间)'
end = 'Wed Mar 03 2022 23:18:00 GMT+0800 (中国标准时间)'

sql_worker = zdz_data(start,end)
a = sql_worker.read_csv()

CPU times: user 184 ms, sys: 37.8 ms, total: 222 ms
Wall time: 286 ms


In [5]:
a

,Unnamed: 0,StationName,county,Town,lat,lon,IIiii,tTime,dFy,fFy,T,Tx,Tn,VV,RR
0,0,洪家,椒江区,洪家街道,28.61806,121.41639,58665,2022-01-25 20:00:00,50,32,101,102,101,5149,0
1,1,大陈,椒江区,大陈镇,28.44600,121.90333,58666,2022-01-25 20:00:00,26,134,92,94,92,4605,0
2,2,坎门,玉环市,坎门街道,28.07750,121.27083,58667,2022-01-25 20:00:00,20,78,93,93,92,5721,0
3,3,洪家,椒江区,洪家街道,28.61806,121.41639,58665,2022-01-25 21:00:00,61,27,99,101,99,4309,0
4,4,大陈,椒江区,大陈镇,28.44600,121.90333,58666,2022-01-25 21:00:00,31,113,92,93,92,5122,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170994,170994,柯思,仙居县,田市镇,28.66389,120.67080,K8908,2022-02-10 06:00:00,0,0,54,55,53,-9999,0
170995,170995,淡竹,仙居县,淡竹乡,28.67194,120.57060,K8911,2022-02-10 06:00:00,265,3,53,54,53,-9999,0
170996,170996,大陈村雪深,仙居县,埠头镇,28.85028,120.51170,K8931,2022-02-10 06:00:00,-9999,-9999,-9999,-9999,-9999,-9999,0
170997,170997,西陆,仙居县,下各镇,28.82278,120.84830,K8950,2022-02-10 06:00:00,130,8,56,57,56,-9999,0
